In [11]:
import pandas as pd
import re
import os

In [12]:
# Load data
file_path = '/Users/meganwagnersmith/Desktop/LibraryProject/LibraryData/Checkouts_by_Title_2022-2023_20241117.csv'
data = pd.read_csv(file_path).dropna()

/var/folders/n9/nhw5lfp14pz9_5f7p9093sd80000gn/T/ipykernel_92299/178088492.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path).dropna()


In [13]:
# Clean ISBN to only include the first one if multiple
def clean_isbn(isbn):
    if pd.isna(isbn):
        return None
    return str(isbn).split(',')[0].strip()

data['ISBN'] = data['ISBN'].apply(clean_isbn)

In [14]:
# Combine CheckoutYear and CheckoutMonth into a single column
data['CheckoutMonthYear'] = data['CheckoutYear'].astype(str) + '-' + data['CheckoutMonth'].astype(str).str.zfill(2)

# Drop the original CheckoutYear and CheckoutMonth columns
data = data.drop(columns=['CheckoutYear', 'CheckoutMonth'])

In [15]:
# Clean publication year
def clean_publication_year_extended(pub_year):
    if pd.isna(pub_year):
        return None
    years = re.findall(r'\d{4}', str(pub_year))
    return int(min(years)) if years else None

data['PublicationYear'] = data['PublicationYear'].apply(clean_publication_year_extended).astype('Int64')


In [16]:
# Clean and standardize publisher
def clean_publisher(publisher):
    if pd.isna(publisher):
        return None
    publisher_cleaned = re.sub(r'[,\.\s]+$', '', publisher)  # Remove trailing commas, periods, or spaces
    publisher_cleaned = re.sub(r'\b(inc|ltd|llc|company|corp|co|publishers|press)\b', '', publisher_cleaned, flags=re.IGNORECASE)
    publisher_cleaned = re.sub(r'[^\w\s]', '', publisher_cleaned)  # Remove punctuation
    return re.sub(r'\s+', ' ', publisher_cleaned).strip().title()

data['Publisher'] = data['Publisher'].apply(clean_publisher)

In [17]:
# Process genres into a separate table
def clean_and_split_subjects(subjects):
    if pd.isna(subjects):
        return None
    return [genre.strip().lower() for genre in str(subjects).split(',')]

data['CleanedSubjects'] = data['Subjects'].apply(clean_and_split_subjects)
isbn_genres = data[['ISBN', 'CleanedSubjects']].explode('CleanedSubjects').dropna(subset=['CleanedSubjects'])
isbn_genres = isbn_genres.rename(columns={'CleanedSubjects': 'Genre'})

In [18]:
# Save genres table
isbn_genres_file = '/Users/meganwagnersmith/Desktop/LibraryProject/LibraryData/isbn_genres.csv'
isbn_genres.to_csv(isbn_genres_file, index=False)


In [19]:
# Drop Subjects and CleanedSubjects from main data
data = data.drop(columns=['Subjects', 'CleanedSubjects'])

# Create ISBN table
checkout_table = data[['ISBN', 'CheckoutMonthYear', 'Checkouts']].drop_duplicates()
checkout_table_file = '/Users/meganwagnersmith/Desktop/LibraryProject/LibraryData/checkout_table.csv'
checkout_table.to_csv(checkout_table_file, index=False)

In [20]:
# Create Book Details table ensuring unique ISBNs
book_details = data[['ISBN', 'Title', 'Creator', 'Publisher', 'PublicationYear']].drop_duplicates()
book_details_unique = book_details.groupby('ISBN').first().reset_index()  # Ensure unique ISBNs
book_details_unique = book_details_unique.rename(columns={'Creator': 'Author'})
book_details_file = '/Users/meganwagnersmith/Desktop/LibraryProject/LibraryData/book_details.csv'
book_details_unique.to_csv(book_details_file, index=False)